In [1]:
## FDR correction After gene level adjustment
SHORTEN = {
    "Artery-Aorta":"Artery A."     ,
    "Artery-Tibial": "Artery T.",
    "Adipose-Subcutaneous": "Adipose S.",    
    "Adipose-Visceral(Omentum)":"Adipose V.",
    "Brain-Caudate(basalganglia)":"Caudate"   , 
    "Brain-Cerebellum":"Cerebellum",
    "Cells-Transformedfibroblasts": "Fibroblast",
    "Esophagus-Mucosa": "E. Mucosa",
    "Esophagus-Muscularis":"E Muscularis",
    "Heart-LeftVentricle":"Ventricule",
    "Lung": "Lung",
    "Muscle-Skeletal": "Muscle",
    "Nerve-Tibial":"Nerve",
    "Skin-NotSunExposed(Suprapubic)": "Skin Unexposed",
    "Skin-SunExposed(Lowerleg)":"Skin Leg",
    "Thyroid":"Thyroid",
    "WholeBlood": "Blood"
}

tissu = SHORTEN.keys()
#path = "/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/"
path = "/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/"


import pandas as pd
import numpy as np
import subprocess
import collections
def fdrcorrection(tissue):
    print(tissue, ' variants ...')
#Get most signif. variant by gene from linear reg STRs
    LR1=pd.read_csv(path+ tissue+"/Lin_Reg_Out" , '\t')       #/Lin_Reg_OutFin.txt
    LR0 = LR1.sort_values("p.wald").groupby("gene", as_index=False).first()     
    print(LR1.shape, '  to  ', LR0.shape)

#Add counts tests by gene
    counts=pd.DataFrame({'cts' : LR1.groupby(["gene"]).size()})    ## This is the count by genes
    genes = list(LR0['gene'])
    LR0['NTEST']= list(counts.loc[genes]['cts'])
    print (len(counts.loc[genes]['cts']))
    
#Minor adjustment
    #(1) min_pval* #test
    LR0['AD.pval']=LR0['p.wald']*LR0['NTEST']
    #(2) if AD_pval>1 => AD_pval=1
    LR0['AD.pval'][LR0['AD.pval']>1] = 1
    
#Save pval in file and FDR correct
    LR0['AD.pval'].to_csv('pvalues.txt', sep='\n', index=False)
    Tell = subprocess.call("/home/szfeupe/projects/GTEX_eSTRs/gtex-estrs/Scripts/PostRegressionFixes/fdr-correct.r")
    
#FDR corrected... add to dataframe
    Qval=pd.read_csv('/home/szfeupe/projects/GTEX_eSTRs/gtex-estrs/Scripts/PostRegressionFixes/qvalues.txt', sep=' ')
    #print (Qval.shape, '\t', LR0.shape)
    LR0['qvalue']=list(Qval['qvalue'])
    LR0['significant']=list(Qval['significant'])
    
#Header arrangement
    Head=['gene','chrom','str.id','str.start','NTEST','p.wald','AD.pval','qvalue','significant','beta','beta.se']
    Out=LR0[Head]
    Out.to_csv(path+tissue+'/PQValues', sep='\t', index=False)

    S=LR0['AD.pval']
    print(len(S),' total tests... ', len(S[S>=1]) , ' pvalues were reduced to 1')
    print(len(Qval[Qval['qvalue'] <=0.1]),'\t qval<=0.1')
    print(len(Qval[Qval['qvalue'] <=0.05]),'\t qval<=0.05')
    print(len(Qval[Qval['qvalue'] <=0.01]),'\t qval<0.01\n')
#   print('Original linear reg. tests counts\n',LR1.groupby('chrom').size())
#   print('After FDR correction test counts\n',LR0.groupby('chrom').size())    
    return()

#fdrcorrection('Lung/SNP_Analysis')
for T in tissu:                     #[x for x in tissu if 'Brain' not in x and x not in ['WholeBlood','Esophagus-Mucosa','Skin-SunExposed(Lowerleg)','Heart-LeftVentricle','Artery-Tibial','Muscle-Skeletal','Adipose-Visceral(Omentum)','Artery-Aorta','Esophagus-Muscularis','Adipose-Subcutaneous','Cells-Transformedfibroblasts','Thyroid']]:                #tissu:
    fdrcorrection(T+'/SNP_Analysis')
    
##########   Next: We run the code to prepare for heritability analysis (STR+SNP)

Nerve-Tibial/SNP_Analysis  variants ...
(8843380, 13)   to   (16257, 13)
16257


/usr/lib/python3.4/site-packages/ipykernel/__main__.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


16257  total tests...  7480  pvalues were reduced to 1
4678 	 qval<=0.1
4178 	 qval<=0.05
3455 	 qval<0.01

Muscle-Skeletal/SNP_Analysis  variants ...
(8360959, 13)   to   (15455, 13)
15455
15455  total tests...  8102  pvalues were reduced to 1
3454 	 qval<=0.1
3105 	 qval<=0.05
2448 	 qval<0.01

Skin-SunExposed(Lowerleg)/SNP_Analysis  variants ...
(8811928, 13)   to   (16306, 13)
16306
16306  total tests...  8018  pvalues were reduced to 1
4095 	 qval<=0.1
3645 	 qval<=0.05
2886 	 qval<0.01

Cells-Transformedfibroblasts/SNP_Analysis  variants ...
(8186925, 13)   to   (15354, 13)
15354
15354  total tests...  7968  pvalues were reduced to 1
3326 	 qval<=0.1
2993 	 qval<=0.05
2350 	 qval<0.01

Brain-Caudate(basalganglia)/SNP_Analysis  variants ...
(8413929, 13)   to   (16344, 13)
16344
16344  total tests...  11466  pvalues were reduced to 1
761 	 qval<=0.1
635 	 qval<=0.05
398 	 qval<0.01

Esophagus-Muscularis/SNP_Analysis  variants ...
(8622045, 13)   to   (16068, 13)
16068
16068  total

In [ ]:
Tissues done
"Adipose-Subcutaneous","Artery-Tibial"
"Adipose-Visceral(Omentum)","Artery-Aorta"
"Cells-Transformedfibroblasts","WholeBlood"












In [2]:
#           MAKE SURE YOU HAVE THE REGRESSION OUTPUT MERGED FIRST ie all chrs in 1 file
tissu = ['WholeBlood','Cells-Transformedfibroblasts','Muscle-Skeletal','Lung','Adipose-Subcutaneous','Artery-Tibial','Esophagus-Mucosa']

##This code will remove STRs that are homopolymer or fall in 
##least polymorphic sites from linear regression analysis output


import pandas as pd
import numpy as np

def filterregression(Tissu):
    #
    print(Tissu)
    chrs = [i for i in range(1,23)]
    chrs.append('X') ; chrs.append('Y')

    #regression
    Input='/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/'+Tissu
    linreg = pd.read_csv(Input+'/Lin_Reg_Out', sep='\t')
#    linreg = pd.read_csv(Input+'/Lin_Reg_Out_perm', sep='\t')

    #Homopolymers
    strfile= "/storage/szfeupe/Runs/GTEx_estr/Homopolymer_sites.tsv"
    L_pol = pd.read_csv(strfile, sep='\t')

    index=['0']
    Out = pd.DataFrame(index=index, columns=linreg.columns)

    #LR Cleanup
    for i in chrs:
    #Open sites to be removed
        Lpolym = '/storage/szfeupe/Runs/GTEx_estr/Heterozygosity/Heterozygosity_VCF.chr'+str(i)
        Reg_to_remove=pd.read_csv(Lpolym, sep='\t', header=None)
        Reg_to_remove.columns=['CHROM','START','END','NumAlleleLenghts','NumAlleles','Het.length','Het.genotype']
        Homopoly = L_pol.loc[L_pol['chrom']=='chr'+str(i)]

    #   Remove Homoploymers
        LIN = linreg.loc[linreg['chrom']=='chr'+str(i)]
        LIN0 = LIN.loc[LIN['str.start'].isin(list(Homopoly['str.start']))==False]

    #   Remove least polymorphic
        Reg=Reg_to_remove.loc[Reg_to_remove['Het.length']<=0.3]
        FIN = LIN0.loc[LIN0['str.start'].isin(list(Reg['START']))==False]

    #   Sum it up and outout   
        frames = [Out, FIN]
        Out = pd.concat(frames)

        print(LIN.shape, LIN0.shape, FIN.shape, 'End')

    Out1=Out.drop('0')
    Out1=Out1.drop('Unnamed: 0', 1)
    Out1.to_csv(Input+'/Lin_Reg_OutFin.txt', sep='\t', index=False)
    #Out1.to_csv(Input+'/Lin_Reg_Out_perm_Fin.txt', sep='\t', index=False)

for T in tissu:
    filterregression(T+'/SNP_Analysis')
    
####After this Script, plot the qqplot and then FDR correction below

WholeBlood/SNP_Analysis
(902066, 13) (902035, 13) (901976, 13) End
(630028, 13) (629982, 13) (629958, 13) End
(577360, 13) (577338, 13) (577302, 13) End
(429113, 13) (429097, 13) (429061, 13) End
(438934, 13) (438903, 13) (438879, 13) End
(637134, 13) (637120, 13) (637075, 13) End
(487894, 13) (487871, 13) (487852, 13) End
(371171, 13) (371163, 13) (371144, 13) End
(382213, 13) (382195, 13) (382175, 13) End
(422058, 13) (422046, 13) (422011, 13) End
(508536, 13) (508522, 13) (508486, 13) End
(503939, 13) (503926, 13) (503908, 13) End
(186898, 13) (186897, 13) (186890, 13) End
(309828, 13) (309798, 13) (309779, 13) End
(297667, 13) (297655, 13) (297637, 13) End
(413588, 13) (413585, 13) (413567, 13) End
(486064, 13) (486019, 13) (486002, 13) End
(173099, 13) (173095, 13) (173088, 13) End
(697074, 13) (697060, 13) (697032, 13) End
(263576, 13) (263552, 13) (263532, 13) End
(117778, 13) (117774, 13) (117772, 13) End
(238224, 13) (238204, 13) (238199, 13) End
(176410, 13) (176399, 13) (176

In [4]:
tissu = ['WholeBlood','Cells-Transformedfibroblasts','Muscle-Skeletal','Lung','Adipose-Subcutaneous','Artery-Tibial','Esophagus-Mucosa']

##Simpler version of FDR correction 
import pandas as pd
import numpy as np
import subprocess
import collections
def fdrcorrection(tissue):
    print(tissue, ' variants ...')
#Get most signif. variant by gene from linear reg STRs
    LR1=pd.read_csv("/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/"+ tissue+"/Lin_Reg_OutFin.txt" , '\t')
    LR0 = LR1.sort_values("p.wald").groupby("gene", as_index=False).first()     
    print(LR1.shape, '  to  ', LR0.shape)

#Add counts tests by gene
    counts=pd.DataFrame({'cts' : LR1.groupby(["gene"]).size()})    ## This is the count by genes
    genes = list(LR0['gene'])
    LR0['NTEST']= list(counts.loc[genes]['cts'])
    print (len(counts.loc[genes]['cts']))
    
#Minor adjustment
    #(1) min_pval* #test
    LR0['AD.pval']=LR0['p.wald']*LR0['NTEST']
    #(2) if AD_pval>1 => AD_pval=1
    LR0['AD.pval'][LR0['AD.pval']>1] = 1
    
#Save pval in file and FDR correct
    LR0['AD.pval'].to_csv('pvalues.txt', sep='\n', index=False)
    Tell = subprocess.call("/home/szfeupe/projects/GTEX_eSTRs/gtex-estrs/Scripts/PostRegressionFixes/fdr-correct.r")
    
#FDR corrected... add to dataframe
    Qval=pd.read_csv('/home/szfeupe/projects/GTEX_eSTRs/gtex-estrs/Scripts/PostRegressionFixes/qvalues.txt', sep=' ')
    #print (Qval.shape, '\t', LR0.shape)
    LR0['qvalue']=list(Qval['qvalue'])
    LR0['significant']=list(Qval['significant'])
    
#Header arrangement
    Head=['gene','chrom','str.id','str.start','NTEST','p.wald','AD.pval','qvalue','significant','beta','beta.se']
    Out=LR0[Head]
    Out.to_csv("/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/"+tissue+'/PQValues.txt', sep='\t', index=False)

    S=LR0['AD.pval']
    print(len(S),' total tests... ', len(S[S>=1]) , ' pvalues were reduced to 1')
    print(len(Qval[Qval['qvalue'] <=0.1]),'\t qval<=0.1')
    print(len(Qval[Qval['qvalue'] <=0.05]),'\t qval<=0.05')
    print(len(Qval[Qval['qvalue'] <=0.01]),'\t qval<0.01\n')
#   print('Original linear reg. tests counts\n',LR1.groupby('chrom').size())
#   print('After FDR correction test counts\n',LR0.groupby('chrom').size())    
    return()

#fdrcorrection('Lung/SNP_Analysis')
for T in tissu:
    fdrcorrection(T)#+'/SNP_Analysis')
    
##########   Next: We run the code to prepare for heritability analysis (STR+SNP)

WholeBlood  variants ...
(209131, 12)   to   (14845, 12)
14845


/usr/lib/python3.4/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


14845  total tests...  5015  pvalues were reduced to 1

630 	 qval<=0.1
462 	 qval<=0.05
261 	 qval<0.01

Cells-Transformedfibroblasts  variants ...
(216353, 12)   to   (15362, 12)
15362
15362  total tests...  4885  pvalues were reduced to 1

955 	 qval<=0.1
703 	 qval<=0.05
395 	 qval<0.01

Muscle-Skeletal  variants ...
(215860, 12)   to   (15275, 12)
15275
15275  total tests...  5546  pvalues were reduced to 1

304 	 qval<=0.1
205 	 qval<=0.05
0 	 qval<0.01

Lung  variants ...
(225784, 12)   to   (15711, 12)
15711
15711  total tests...  5673  pvalues were reduced to 1

378 	 qval<=0.1
264 	 qval<=0.05
0 	 qval<0.01

Adipose-Subcutaneous  variants ...
(225770, 12)   to   (15918, 12)
15918
15918  total tests...  5751  pvalues were reduced to 1

376 	 qval<=0.1
287 	 qval<=0.05
115 	 qval<0.01

Artery-Tibial  variants ...
(221263, 12)   to   (15584, 12)
15584
15584  total tests...  5360  pvalues were reduced to 1

524 	 qval<=0.1
383 	 qval<=0.05
196 	 qval<0.01

Esophagus-Mucosa  varia

In [ ]:
WholeBlood  variants ...
(209131, 12)   to   (14845, 12)
14845
/usr/lib/python3.4/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
14845  total tests...  5015  pvalues were reduced to 1

630 	 qval<=0.1
462 	 qval<=0.05
261 	 qval<0.01

Cells-Transformedfibroblasts  variants ...
(216353, 12)   to   (15362, 12)
15362
15362  total tests...  4885  pvalues were reduced to 1

955 	 qval<=0.1
703 	 qval<=0.05
395 	 qval<0.01

Muscle-Skeletal  variants ...
(215860, 12)   to   (15275, 12)
15275
15275  total tests...  5546  pvalues were reduced to 1

304 	 qval<=0.1
205 	 qval<=0.05
0 	 qval<0.01

Lung  variants ...
(225784, 12)   to   (15711, 12)
15711
15711  total tests...  5673  pvalues were reduced to 1

378 	 qval<=0.1
264 	 qval<=0.05
0 	 qval<0.01

Adipose-Subcutaneous  variants ...
(225770, 12)   to   (15918, 12)
15918
15918  total tests...  5751  pvalues were reduced to 1

376 	 qval<=0.1
287 	 qval<=0.05
115 	 qval<0.01

Artery-Tibial  variants ...
(221263, 12)   to   (15584, 12)
15584
15584  total tests...  5360  pvalues were reduced to 1

524 	 qval<=0.1
383 	 qval<=0.05
196 	 qval<0.01

Esophagus-Mucosa  variants ...
(227378, 12)   to   (16111, 12)
16111
16111  total tests...  5582  pvalues were reduced to 1

475 	 qval<=0.1
330 	 qval<=0.05
128 	 qval<0.01

In [ ]:

​
